In [1]:
%%time
#dict.opcorpora.txt
#sample.txt
from pprint import pp
lines = []
with open("dict.opcorpora.txt", "r", encoding='utf-8') as f:
    line = f.readline()
    while line:
        if line != '\n' and not line[0].isdigit(): 
            lines.append(line[:-1])
        line = f.readline()

pp(len(lines))

5138879
CPU times: total: 5.95 s
Wall time: 6.08 s


In [4]:
import re
from tqdm import tqdm

inf = {
    'NOUN' : (['sing', 'nomn'], ["Abbr", "Name", "Surn", "Patr", "Geox", "Orgn", "Trad", "Erro"]),
    'INFN' : ([], ["Erro"]),                    
    'ADJF' : (['masc', 'sing', 'nomn'], ["Erro"]),        
    'PRTF' : (['masc', 'sing', 'nomn'], ["Erro"]), 
    'NUMR' : (['nomn'], ["Erro"])
}
    
def approve(line: str) -> bool:
    features = re.split('\t|,| ', line)
    if not re.match("^[А-ЯЁ]*$", features[0]):
        return False
    post = features[1]
    if post in inf:
        return check_features(post, features[2:])
    return False

def check_features(post: str, feats: list) -> bool:
    if ((all(map(lambda f: f in feats, inf[post][0])) if inf[post][0] else True) 
        and (all(map(lambda f: f not in feats, inf[post][1])) if inf[post][1] else True)):
        return True
    return False


CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
%%time
filtered = list(filter(approve, tqdm(lines, position=0, leave=True)))
print(len(filtered))

100%|██████████| 5138879/5138879 [00:21<00:00, 235059.43it/s]

208995
CPU times: total: 2.08 s
Wall time: 21.9 s


In [13]:
%%time
just_words = list(map(lambda line: re.split('\t|,| ', line)[0], tqdm(filtered, position=0, leave=True)))
print(len(just_words))

100%|██████████| 208995/208995 [00:00<00:00, 404442.26it/s]

208995
CPU times: total: 141 ms
Wall time: 520 ms


In [14]:
%%time
no_dupl = list(dict.fromkeys(just_words))
print(len(no_dupl))

203425
CPU times: total: 62.5 ms
Wall time: 53.1 ms


In [18]:
with open("fitered_opencorpora.txt", "w+", encoding="utf-8") as nf:
    nf.writelines(f"{line}\n" for line in no_dupl)